

## Naive Bayes on Political Text

### Conor Fitzpatrick

In this notebook we use Naive Bayes to explore and classify political data. See the `README.md` for full details. You can download the required DB from the shared dropbox or from blackboard

In [1]:
import sqlite3
import nltk
import random
import numpy as np
from collections import Counter, defaultdict
import sqlite3
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


# Feel free to include your text patterns functions
#from text_functions_solutions import clean_tokenize, get_patterns

/Users/cfitzpatrick/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
convention_db = sqlite3.connect("/Users/cfitzpatrick/Downloads/2020_Conventions (3).db")
convention_cur = convention_db.cursor()

In [5]:
import sqlite3
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Download NLTK data
nltk.download('punkt')
nltk.download('stopwords')

# Function to clean and tokenize text
def clean_tokenize(text):
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text.lower())
    cleaned_tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    return ' '.join(cleaned_tokens)

# Path to the database
db_path = "/Users/cfitzpatrick/Downloads/nb-assignment-data/2020_Conventions.db"


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/cfitzpatrick/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/cfitzpatrick/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
# Connect to the database
convention_db = sqlite3.connect(db_path)
convention_cur = convention_db.cursor()


In [7]:
# Query to list all tables
tables_query = "SELECT name FROM sqlite_master WHERE type='table';"
convention_cur.execute(tables_query)
tables = convention_cur.fetchall()
print("Tables in the database:", tables)


Tables in the database: [('conventions',)]


In [8]:
for table in tables:
    table_name = table[0]
    schema_query = f"PRAGMA table_info({table_name});"
    convention_cur.execute(schema_query)
    schema = convention_cur.fetchall()
    print(f"Schema of {table_name} table:", schema)


Schema of conventions table: [(0, 'party', 'TEXT', 0, None, 0), (1, 'night', 'INTEGER', 0, None, 0), (2, 'speaker', 'TEXT', 0, None, 0), (3, 'speaker_count', 'INTEGER', 0, None, 0), (4, 'time', 'TEXT', 0, None, 0), (5, 'text', 'TEXT', 0, None, 0), (6, 'text_len', 'TEXT', 0, None, 0), (7, 'file', 'TEXT', 0, None, 0)]


### Part 1: Exploratory Naive Bayes

We'll first build a NB model on the convention data itself, as a way to understand what words distinguish between the two parties. This is analogous to what we did in the "Comparing Groups" class work. First, pull in the text 
for each party and prepare it for use in Naive Bayes.  

In [13]:
import sqlite3
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')

def clean_tokenize(text):
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text.lower())
    cleaned_tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    return ' '.join(cleaned_tokens)

convention_data = []

# fill this list up with items that are themselves lists. The 
# first element in the sublist should be the cleaned and tokenized
# text in a single string. As part of your cleaning process,
# remove the stopwords from the text. The second element of the sublist
# should be the party.

query = '''
SELECT text, party FROM conventions
'''

# Connect to the database
convention_db = sqlite3.connect("/Users/cfitzpatrick/Downloads/2020_Conventions (3).db")
convention_cur = convention_db.cursor()

query_results = convention_cur.execute(query)

for row in query_results:
    # Clean and tokenize the text
    cleaned_text = clean_tokenize(row[0])
    # Store the results in convention_data
    convention_data.append([cleaned_text, row[1]])



[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/cfitzpatrick/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/cfitzpatrick/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# def clean_tokenize(text):
#     stop_words = set(stopwords.words('english'))
#     tokens = word_tokenize(text.lower())
#     cleaned_tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
#     return ' '.join(cleaned_tokens)


# convention_data = []

# # fill this list up with items that are themselves lists. The 
# # first element in the sublist should be the cleaned and tokenized
# # text in a single string. As part of your cleaning process,
# # remove the stopwords from the text. The second element of the sublist
# # should be the party. 

# query = '''
# SELECT text, party FROM speeches
# '''

# query_results = convention_cur.execute(query)

# # for row in query_results :
# #     # store the results in convention_data
# #     pass # remove this
    


Let's look at some random entries and see if they look right. 

In [12]:
random.choices(convention_data,k=5)

[['unified group around clean energy strategy', 'Democratic'],
 ['meg whitman longtime republican longtime ceo let tell donald trump clue run business let alone economy joe biden hand plan strengthen economy working people small business owners choice simple joe',
  'Democratic'],
 ['every single family america affected cancer close', 'Democratic'],
 ['also moved joe work going back forth leader wrote violence women act enacted assault weapons ban vice president implemented recovery act brought country back great recessions championed affordable care act protecting millions americans preexisting conditions spent decades promoting american values interests around world',
  'Democratic'],
 ['today beautiful daughter hope thriving old crystal fast approaching three years recovery dear friend constant inspiration others hold special place heart facing opioid addiction enormously grateful president leadership fighting deadly enemy efforts turning tide crisis addiction president trump declar

If that looks good, we now need to make our function to turn these into features. In my solution, I wanted to keep the number of features reasonable, so I only used words that occur at least `word_cutoff` times. Here's the code to test that if you want it. 

In [14]:
word_cutoff = 5

tokens = [w for t, p in convention_data for w in t.split()]

word_dist = nltk.FreqDist(tokens)

feature_words = set()

for word, count in word_dist.items() :
    if count > word_cutoff :
        feature_words.add(word)
        
print(f"With a word cutoff of {word_cutoff}, we have {len(feature_words)} as features in the model.")

With a word cutoff of 5, we have 2236 as features in the model.


In [16]:
def conv_features(text,fw) :
    """Given some text, this returns a dictionary holding the
       feature words.
       
       Args: 
            * text: a piece of text in a continuous string. Assumes
            text has been cleaned and case folded.
            * fw: the *feature words* that we're considering. A word 
            in `text` must be in fw in order to be returned. This 
            prevents us from considering very rarely occurring words.
        
       Returns: 
            A dictionary with the words in `text` that appear in `fw`. 
            Words are only counted once. 
            If `text` were "quick quick brown fox" and `fw` = {'quick','fox','jumps'},
            then this would return a dictionary of 
            {'quick' : True,
             'fox' :    True}
        
    """
    
    # Your code here
    
        # Initialize an empty dictionary to store the features
    ret_dict = dict()
    
    # Split the text into words
    words = text.split()
    
    # Iterate over each word in the text
    for word in words:
        # If the word is in the feature words set, add it to the dictionary
        if word in fw:
            ret_dict[word] = True
    
    return ret_dict



In [17]:
assert(len(feature_words)>0)
assert(conv_features("donald is the president",feature_words)==
       {'donald':True,'president':True})
assert(conv_features("some people in america are citizens",feature_words)==
                     {'people':True,'america':True,"citizens":True})

Now we'll build our feature set. Out of curiosity I did a train/test split to see how accurate the classifier was, but we don't strictly need to since this analysis is exploratory. 

In [18]:
featuresets = [(conv_features(text,feature_words), party) for (text, party) in convention_data]

In [19]:
random.seed(20220507)
random.shuffle(featuresets)

test_size = 500

In [20]:
test_set, train_set = featuresets[:test_size], featuresets[test_size:]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.494


In [21]:
classifier.show_most_informative_features(25)

Most Informative Features
                   china = True           Republ : Democr =     27.1 : 1.0
                   votes = True           Democr : Republ =     23.8 : 1.0
             enforcement = True           Republ : Democr =     21.5 : 1.0
                 destroy = True           Republ : Democr =     19.2 : 1.0
                freedoms = True           Republ : Democr =     18.2 : 1.0
                 climate = True           Democr : Republ =     17.8 : 1.0
                supports = True           Republ : Democr =     17.1 : 1.0
                   crime = True           Republ : Democr =     16.1 : 1.0
                   media = True           Republ : Democr =     15.8 : 1.0
                 beliefs = True           Republ : Democr =     13.0 : 1.0
               countries = True           Republ : Democr =     13.0 : 1.0
                 defense = True           Republ : Democr =     13.0 : 1.0
                  defund = True           Republ : Democr =     13.0 : 1.0

Write a little prose here about what you see in the classifier. Anything odd or interesting?

### My Observations

The accuracy on the test set is not very good at 49%. This is basically about what I would expect from randomly guessing. The most important features show some interesting insights. For Republicans, China is at the top of the list. This makes sense to me, since China is often seen as a big economical threat by many Republicans. The  top words for Republicans are not unexpected, like enforcement and destroy and freedoms. For Democrats, climate is the most common, which certainly makes sense, as the climate is a huge part of many Democrat platforms. Abraham being used by Democrats is interesting, maybe they want to identify with Lincoln? 



## Part 2: Classifying Congressional Tweets

In this part we apply the classifer we just built to a set of tweets by people running for congress
in 2018. These tweets are stored in the database `congressional_data.db`. That DB is funky, so I'll
give you the query I used to pull out the tweets. Note that this DB has some big tables and 
is unindexed, so the query takes a minute or two to run on my machine.

In [26]:
cong_db = sqlite3.connect("/Users/cfitzpatrick/Downloads/nb-assignment-data (1)/congressional_data.db")
cong_cur = cong_db.cursor()

In [27]:
results = cong_cur.execute(
        '''
           SELECT DISTINCT 
                  cd.candidate, 
                  cd.party,
                  tw.tweet_text
           FROM candidate_data cd 
           INNER JOIN tweets tw ON cd.twitter_handle = tw.handle 
               AND cd.candidate == tw.candidate 
               AND cd.district == tw.district
           WHERE cd.party in ('Republican','Democratic') 
               AND tw.tweet_text NOT LIKE '%RT%'
        ''')

results = list(results) # Just to store it, since the query is time consuming

In [28]:
tweet_data = []

# Now fill up tweet_data with sublists like we did on the convention speeches.
# Note that this may take a bit of time, since we have a lot of tweets.


In [33]:
# Prepare the tweet data for classification
tweet_data = []

for candidate, party, tweet_text in results:
    cleaned_text = clean_tokenize(str(tweet_text))  # Ensure tweet_text is treated as a string
    tweet_data.append([cleaned_text, party])

# Close the database connection
cong_db.close()

# Check some random tweet entries
random_tweets = random.sample(tweet_data, 5)
for tweet in random_tweets:
    print(tweet)

# Feature sets for tweets
tweet_featuresets = [(conv_features(text, feature_words), party) for (text, party) in tweet_data]

# Take a random sample of tweets to see how our classifier does
random.seed(20220507)
random.shuffle(tweet_featuresets)

test_size = 500
test_set, train_set = tweet_featuresets[:test_size], tweet_featuresets[test_size:]
tweet_classifier = nltk.NaiveBayesClassifier.train(train_set)

print("Classifier accuracy on tweet test set:", nltk.classify.accuracy(tweet_classifier, test_set))

# Show the most informative features
tweet_classifier.show_most_informative_features(25)

['pauldavisks calls amnesty illegal immigrants calls grant full citizenship remainred', 'Republican']
['first time long time see budget language vote check rep homepage', 'Republican']
['al green selected ranking member financial services subcommittee oversight investigations http', 'Democratic']
['representative james clyburn joining today https', 'Democratic']
['b photo thanks shipbuilders crew minnesota ssn hiindustries va hrva nn usnavy http', 'Democratic']


There are a lot of tweets here. Let's take a random sample and see how our classifer does. I'm guessing it won't be too great given the performance on the convention speeches...

In [31]:
random.seed(20201014)

tweet_data_sample = random.choices(tweet_data,k=10)

In [34]:
# Sample 5 random tweets from tweet_data for checking the classifier
tweet_data_sample = random.sample(tweet_data, 5)

for tweet, party in tweet_data_sample:
    # Extract features from the tweet
    features = conv_features(tweet, feature_words)
    
    # Estimate the party using the classifier
    estimated_party = tweet_classifier.classify(features)
    
    print(f"Here's our (cleaned) tweet: {tweet}")
    print(f"Actual party is {party} and our classifier says {estimated_party}.")
    print("")

Here's our (cleaned) tweet: facebook stay date latest campaign https
Actual party is Democratic and our classifier says Democratic.

Here's our (cleaned) tweet: voted protect middle class economic recovery
Actual party is Democratic and our classifier says Democratic.

Here's our (cleaned) tweet: offer thoughts prayers victims families students faculty taft hs entire taft community
Actual party is Republican and our classifier says Democratic.

Here's our (cleaned) tweet: fathers day guys work hard put families first dad https
Actual party is Republican and our classifier says Democratic.

Here's our (cleaned) tweet: drag friends vote willhaskellct bring remarkable new talent energy fight southwest connecticut future american politics indeed https
Actual party is Democratic and our classifier says Democratic.



In [35]:

# for tweet, party in tweet_data_sample :
#     estimated_party = 'Gotta fill this in'
#     # Fill in the right-hand side above with code that estimates the actual party
    
#     print(f"Here's our (cleaned) tweet: {tweet}")
#     print(f"Actual party is {party} and our classifer says {estimated_party}.")
#     print("")
    

Now that we've looked at it some, let's score a bunch and see how we're doing.

In [38]:
from collections import defaultdict
import random

# dictionary of counts by actual party and estimated party. 
# first key is actual, second is estimated
parties = ['Republican', 'Democratic']
results = defaultdict(lambda: defaultdict(int))

for p in parties:
    for p1 in parties:
        results[p][p1] = 0

num_to_score = 10000
random.shuffle(tweet_data)

for idx, tp in enumerate(tweet_data):
    tweet, party = tp    
    # Extract features from the tweet
    features = conv_features(tweet, feature_words)
    
    # Get the estimated party using the classifier
    estimated_party = tweet_classifier.classify(features)
    
    results[party][estimated_party] += 1
    
    if idx > num_to_score: 
        break

# Let's print out the results
for actual_party in parties:
    for estimated_party in parties:
        print(f"Actual: {actual_party}, Estimated: {estimated_party}, Count: {results[actual_party][estimated_party]}")


Actual: Republican, Estimated: Republican, Count: 1114
Actual: Republican, Estimated: Democratic, Count: 3262
Actual: Democratic, Estimated: Republican, Count: 463
Actual: Democratic, Estimated: Democratic, Count: 5163


In [42]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Calculate the stats for easier analysis
# Given counts
true_positives_rep = 1114
false_positives_rep = 463
false_negatives_rep = 3262
true_negatives_rep = 5163

true_positives_dem = 5163
false_positives_dem = 3262
false_negatives_dem = 463
true_negatives_dem = 1114

# Confusion matrix
conf_matrix = [
    [true_positives_rep, false_negatives_rep],
    [false_positives_rep, true_positives_dem]
]

# Actual and predicted labels based on the confusion matrix
actual_labels = ['Republican'] * (true_positives_rep + false_negatives_rep) + ['Democratic'] * (false_positives_rep + true_positives_dem)
predicted_labels = ['Republican'] * true_positives_rep + ['Democratic'] * false_negatives_rep + ['Republican'] * false_positives_rep + ['Democratic'] * true_positives_dem

# Calculate the overall accuracy
accuracy = accuracy_score(actual_labels, predicted_labels)

# Calculate precision, recall, and F1-score for each class
precision_rep = precision_score(actual_labels, predicted_labels, pos_label='Republican')
recall_rep = recall_score(actual_labels, predicted_labels, pos_label='Republican')
f1_score_rep = f1_score(actual_labels, predicted_labels, pos_label='Republican')

precision_dem = precision_score(actual_labels, predicted_labels, pos_label='Democratic')
recall_dem = recall_score(actual_labels, predicted_labels, pos_label='Democratic')
f1_score_dem = f1_score(actual_labels, predicted_labels, pos_label='Democratic')

# Print the results
print(f"Confusion Matrix:\n{conf_matrix}")
print(f"Overall Accuracy: {accuracy:.4f}")
print(f"Republican Precision: {precision_rep:.4f}")
print(f"Republican Recall: {recall_rep:.4f}")
print(f"Republican F1-score: {f1_score_rep:.4f}")
print(f"Democratic Precision: {precision_dem:.4f}")
print(f"Democratic Recall: {recall_dem:.4f}")
print(f"Democratic F1-score: {f1_score_dem:.4f}")



Confusion Matrix:
[[1114, 3262], [463, 5163]]
Overall Accuracy: 0.6276
Republican Precision: 0.7064
Republican Recall: 0.2546
Republican F1-score: 0.3743
Democratic Precision: 0.6128
Democratic Recall: 0.9177
Democratic F1-score: 0.7349


### Reflections

The Naive Bays accuracy overall was 62.7%, which is not particularly high. It seems that it is better able to predict Democrat tweets than Republicans, with an F1-score for dems at 73%, and only 37% for Republicans. F1-score is a balance of precision and recall, so it can be a good metric. What we see here is that the classifier really struggles to accurately identify republican tweets, and given the fact that it only has two options, these results are not particularly impressive. It is interesting to see the various words that were brough back in the feature importances. China being the most important for Republicans makes a lot of sense, and Democrats using words like Climate also makes a lot of sense. 

Going forward, we could try to do more digging in to the data to better understand what is going on, and see if we can find trends specifically within the Republican tweets that might help the overall accuracy. It is possible there are some differences in the collection of Republican or Democrat tweets, or some other factors we are not aware of. 

